# BDNS Convocatorias Analysis

This notebook provides tools to filter and search through the scraped BDNS convocatorias data.


In [ ]:
import pandas as pd
from pathlib import Path
from typing import Optional, List, Union
from datetime import datetime

# Data directory
DATA_DIR = Path("data")  # Relative path to data directory


## Data Loading Functions


In [63]:
def load_all_data(remove_duplicates: bool = True) -> pd.DataFrame:
    """
    Load all BDNS data from all years.
    
    Args:
        remove_duplicates: If True, removes duplicate BDNS codes
    
    Returns:
        Combined DataFrame with all years
    """
    parquet_files = list(DATA_DIR.glob("bdns_*.parquet"))
    
    if not parquet_files:
        print(f"No data files found in {DATA_DIR}")
        return pd.DataFrame()
    
    dfs = []
    for file in parquet_files:
        df = pd.read_parquet(file)
        dfs.append(df)
        print(f"Loaded {len(df)} rows from {file.name}")
    
    combined = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows before cleaning: {len(combined):,}")
    
    if remove_duplicates:
        combined = clean_duplicates(combined)
    
    return combined


def load_data_by_year(year: Union[int, List[int]], remove_duplicates: bool = True) -> pd.DataFrame:
    """
    Load BDNS data for specific year(s).
    
    Args:
        year: Single year or list of years
        remove_duplicates: If True, removes duplicate BDNS codes
    
    Returns:
        DataFrame with data for specified year(s)
    """
    if isinstance(year, int):
        years = [year]
    else:
        years = year
    
    dfs = []
    for y in years:
        file = DATA_DIR / f"bdns_{y}.parquet"
        if file.exists():
            df = pd.read_parquet(file)
            dfs.append(df)
            print(f"Loaded {len(df)} rows from {file.name}")
        else:
            print(f"No data found for year {y}")
    
    if dfs:
        combined = pd.concat(dfs, ignore_index=True)
        print(f"\nTotal rows before cleaning: {len(combined):,}")
        
        if remove_duplicates:
            combined = clean_duplicates(combined)
        
        return combined
    
    return pd.DataFrame()


def get_available_years() -> List[int]:
    """
    Get list of years with available data.
    
    Returns:
        Sorted list of years
    """
    parquet_files = list(DATA_DIR.glob("bdns_*.parquet"))
    years = []
    
    for file in parquet_files:
        try:
            year = int(file.stem.split('_')[1])
            years.append(year)
        except:
            pass
    
    return sorted(years)


def clean_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicate BDNS entries from the dataset.
    Keeps the last occurrence of each BDNS code.
    
    Args:
        df: DataFrame to clean
    
    Returns:
        Cleaned DataFrame with unique BDNS codes
    """
    initial_count = len(df)
    
    # Remove duplicates based on BDNS code only (keep last occurrence)
    df_clean = df.drop_duplicates(subset=['codigoBDNS'], keep='last')
    
    removed_count = initial_count - len(df_clean)
    print(f"Removed {removed_count:,} duplicate rows")
    print(f"Total rows after cleaning: {len(df_clean):,}")
    print(f"Unique BDNS codes: {df_clean['codigoBDNS'].nunique():,}")
    
    return df_clean.reset_index(drop=True)


## Data Cleaning

The dataset automatically removes duplicate BDNS codes when loading.

In [64]:
# Load data with automatic duplicate removal (default)
# df = load_all_data(remove_duplicates=True)

# Or load without removing duplicates and clean manually later
# df = load_all_data(remove_duplicates=False)
# df = clean_duplicates(df)  # Removes duplicates, keeps one row per BDNS code

## Filtering and Search Functions


In [65]:
def filter_by_bdns(df: pd.DataFrame, bdns_number: Union[str, int, List]) -> pd.DataFrame:
    """
    Filter by BDNS number(s).
    
    Args:
        df: DataFrame to filter
        bdns_number: Single BDNS number or list of numbers
    
    Returns:
        Filtered DataFrame
    """
    if isinstance(bdns_number, (list, tuple)):
        bdns_list = [str(b) for b in bdns_number]
        return df[df['codigoBDNS'].isin(bdns_list)]
    else:
        return df[df['codigoBDNS'] == str(bdns_number)]


def search_descripcion(df: pd.DataFrame, search_term: str, case_sensitive: bool = False) -> pd.DataFrame:
    """
    Search for terms in descripcion field.
    
    Args:
        df: DataFrame to search
        search_term: Term to search for
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        Filtered DataFrame
    """
    if case_sensitive:
        mask = df['descripcion'].str.contains(search_term, na=False)
    else:
        mask = df['descripcion'].str.contains(search_term, case=False, na=False)
    
    return df[mask]


def filter_by_date_range(df: pd.DataFrame, 
                         date_field: str = 'fechaRecepcion',
                         start_date: Optional[str] = None,
                         end_date: Optional[str] = None) -> pd.DataFrame:
    """
    Filter by date range.
    
    Args:
        df: DataFrame to filter
        date_field: Field to filter on ('fechaRecepcion', 'fechaInicioSolicitud', 'fechaFinSolicitud')
        start_date: Start date (YYYY-MM-DD format)
        end_date: End date (YYYY-MM-DD format)
    
    Returns:
        Filtered DataFrame
    """
    df = df.copy()
    df[date_field] = pd.to_datetime(df[date_field], errors='coerce')
    
    if start_date:
        df = df[df[date_field] >= start_date]
    
    if end_date:
        df = df[df[date_field] <= end_date]
    
    return df


def filter_by_institution(df: pd.DataFrame, 
                          nivel1: Optional[str] = None,
                          nivel2: Optional[str] = None,
                          case_sensitive: bool = False) -> pd.DataFrame:
    """
    Filter by institution (organo).
    
    Args:
        df: DataFrame to filter
        nivel1: Filter by nivel1 (partial match)
        nivel2: Filter by nivel2 (partial match)
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        Filtered DataFrame
    """
    result = df.copy()
    
    if nivel1:
        if case_sensitive:
            result = result[result['organo_nivel1'].str.contains(nivel1, na=False)]
        else:
            result = result[result['organo_nivel1'].str.contains(nivel1, case=False, na=False)]
    
    if nivel2:
        if case_sensitive:
            result = result[result['organo_nivel2'].str.contains(nivel2, na=False)]
        else:
            result = result[result['organo_nivel2'].str.contains(nivel2, case=False, na=False)]
    
    return result


def filter_by_budget(df: pd.DataFrame, 
                     min_budget: Optional[float] = None,
                     max_budget: Optional[float] = None) -> pd.DataFrame:
    """
    Filter by budget (presupuestoTotal).
    
    Args:
        df: DataFrame to filter
        min_budget: Minimum budget
        max_budget: Maximum budget
    
    Returns:
        Filtered DataFrame
    """
    result = df.copy()
    
    if min_budget is not None:
        result = result[result['presupuestoTotal'] >= min_budget]
    
    if max_budget is not None:
        result = result[result['presupuestoTotal'] <= max_budget]
    
    return result


def filter_by_region(df: pd.DataFrame, region: str, case_sensitive: bool = False) -> pd.DataFrame:
    """
    Filter by region.
    
    Args:
        df: DataFrame to filter
        region: Region to filter by (partial match)
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        Filtered DataFrame
    """
    if case_sensitive:
        return df[df['region_descripcion'].str.contains(region, na=False)]
    else:
        return df[df['region_descripcion'].str.contains(region, case=False, na=False)]


def filter_by_sector(df: pd.DataFrame, 
                     sector_desc: Optional[str] = None,
                     sector_code: Optional[str] = None,
                     case_sensitive: bool = False) -> pd.DataFrame:
    """
    Filter by sector.
    
    Args:
        df: DataFrame to filter
        sector_desc: Sector description (partial match)
        sector_code: Sector code (exact match)
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        Filtered DataFrame
    """
    result = df.copy()
    
    if sector_desc:
        if case_sensitive:
            result = result[result['sector_descripcion'].str.contains(sector_desc, na=False)]
        else:
            result = result[result['sector_descripcion'].str.contains(sector_desc, case=False, na=False)]
    
    if sector_code:
        result = result[result['sector_codigo'] == sector_code]
    
    return result


def filter_by_tipo_convocatoria(df: pd.DataFrame, tipo: str, case_sensitive: bool = False) -> pd.DataFrame:
    """
    Filter by tipo de convocatoria.
    
    Args:
        df: DataFrame to filter
        tipo: Type to filter by (partial match)
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        Filtered DataFrame
    """
    if case_sensitive:
        return df[df['tipoConvocatoria'].str.contains(tipo, na=False)]
    else:
        return df[df['tipoConvocatoria'].str.contains(tipo, case=False, na=False)]


def filter_by_status(df: pd.DataFrame, abierto: bool) -> pd.DataFrame:
    """
    Filter by open/closed status.
    
    Args:
        df: DataFrame to filter
        abierto: True for open, False for closed
    
    Returns:
        Filtered DataFrame
    """
    return df[df['abierto'] == abierto]


## Summary and Statistics Functions


In [88]:
def get_summary(df: pd.DataFrame) -> None:
    """
    Muestra estadísticas resumen del dataset en español.
    Incluye análisis separado por año.
    
    Args:
        df: DataFrame a resumir
    """
    print("=" * 80)
    print("RESUMEN DEL DATASET")
    print("=" * 80)
    print(f"Total de registros: {len(df):,}")
    print(f"Códigos BDNS únicos: {df['codigoBDNS'].nunique():,}")
    print()
    
    # Rango de fechas
    df_temp =df.copy()
    df_temp['fechaRecepcion'] = pd.to_datetime(df_temp['fechaRecepcion'], errors='coerce')
    print(f"Rango de fechas: {df_temp['fechaRecepcion'].min()} a {df_temp['fechaRecepcion'].max()}")
    print()
    
    # Años disponibles
    years = sorted(df['year'].dropna().unique())
    print(f"Años disponibles: {years}")
    print()
    
    # Estadísticas generales de presupuesto
    budget_stats = df['presupuestoTotal'].describe()
    print("ESTADÍSTICAS GENERALES DE PRESUPUESTO")
    print("-" * 80)
    print(f"  Total: €{df['presupuestoTotal'].sum():,.2f}")
    print(f"  Media: €{budget_stats['mean']:,.2f}")
    print(f"  Mediana: €{budget_stats['50%']:,.2f}")
    print(f"  Mínimo: €{budget_stats['min']:,.2f}")
    print(f"  Máximo: €{budget_stats['max']:,.2f}")
    print()
    
    # Estado general
    status_counts = df['abierto'].value_counts()
    print("ESTADO GENERAL")
    print("-" * 80)
    print(f"  Abiertas: {status_counts.get(True, 0):,}")
    print(f"  Cerradas: {status_counts.get(False, 0):,}")
    print()
    
    # Análisis por año
    print("=" * 80)
    print("ANÁLISIS POR AÑO")
    print("=" * 80)
    
    for year in years:
        df_year = df[df['year'] == year]
        if df_year.empty:
            continue
            
        print(f"\n{'=' * 80}")
        print(f"AÑO {year}")
        print(f"{'=' * 80}")
        print(f"Registros: {len(df_year):,}")
        print(f"Códigos BDNS únicos: {df_year['codigoBDNS'].nunique():,}")
        print()
        
        # Rango de fechas del año
        df_year_temp = df_year.copy()
        df_year_temp['fechaRecepcion'] = pd.to_datetime(df_year_temp['fechaRecepcion'], errors='coerce')
        print(f"Rango de fechas: {df_year_temp['fechaRecepcion'].min()} a {df_year_temp['fechaRecepcion'].max()}")
        print()
        
        # Estadísticas de presupuesto del año
        budget_stats_year = df_year['presupuestoTotal'].describe()
        print("Presupuesto:")
        print(f"  Total: €{df_year['presupuestoTotal'].sum():,.2f}")
        print(f"  Media: €{budget_stats_year['mean']:,.2f}")
        print(f"  Mediana: €{budget_stats_year['50%']:,.2f}")
        print(f"  Mínimo: €{budget_stats_year['min']:,.2f}")
        print(f"  Máximo: €{budget_stats_year['max']:,.2f}")
        print()
        
        # Estado del año
        status_counts_year = df_year['abierto'].value_counts()
        print("Estado:")
        print(f"  Abiertas: {status_counts_year.get(True, 0):,}")
        print(f"  Cerradas: {status_counts_year.get(False, 0):,}")
        print()
        
        # Top 5 regiones del año
        print("Top 5 regiones:")
        region_counts_year = df_year['region_descripcion'].value_counts().head(5)
        for i, (region, count) in enumerate(region_counts_year.items(), 1):
            print(f"  {i}. {region}: {count:,}")
        print()
        
        # Top 5 sectores del año جو
        print("Top 5 sectores:")
        sector_counts_year = df_year['sector_descripcion'].value_counts().head(5)
        for i, (sector, count) in enumerate(sector_counts_year.items(), 1):
            print(f"  {i}. {sector}: {count:,}")
        print()
    
    # Top 10 regiones generales
    print("=" * 80)
    print("TOP 10 REGIONES (GENERAL)")
    print("-" * 80)
    region_counts = df['region_descripcion'].value_counts().head(10)
    for i, (region, count) in enumerate(region_counts.items(), 1):
        print(f"  {i}. {region}: {count:,}")
    print()
    
    # Top 10 sectores generales
    print("TOP 10 SECTORES (GENERAL)")
    print("-" * 80)
    sector_counts = df['sector_descripcion'].value_counts().head(10)
    for i, (sector, count) in enumerate(sector_counts.items(), 1):
        print(f"  {i}. {sector}: {count:,}")
    print()
    
    print("=" * 80)


def display_results(df: pd.DataFrame, max_rows: int = 10, columns: Optional[List[str]] = None) -> None:
    """
    Display filtered results in a readable format.
    
    Args:
        df: DataFrame to display
        max_rows: Maximum number of rows to display
        columns: Specific columns to display (None for all)
    """
    if df.empty:
        print("No results found.")
        return
    
    print(f"Found {len(df):,} results\n")
    
    if columns:
        display_df = df[columns].head(max_rows)
    else:
        # Display key columns by default
        key_columns = [
            'codigoBDNS', 'fechaRecepcion', 'descripcion', 'presupuestoTotal',
            'organo_nivel1', 'region_descripcion', 'abierto'
        ]
        available_columns = [col for col in key_columns if col in df.columns]
        display_df = df[available_columns].head(max_rows)
    
    # Set pandas display options for better readability
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.width', None)
    
    display(display_df)
    
    if len(df) > max_rows:
        print(f"\n... and {len(df) - max_rows:,} more rows")


## Example Usage

Below are examples of how to use the functions to load and filter data.


In [89]:
# Check available years
available_years = get_available_years()
print(f"Available years: {available_years}")


Available years: [2023, 2024]


In [90]:
# Load all data
df = load_all_data()


Loaded 35942 rows from bdns_2023.parquet
Loaded 10072 rows from bdns_2024.parquet

Total rows before cleaning: 46,014
Removed 0 duplicate rows
Total rows after cleaning: 46,014
Unique BDNS codes: 46,014


In [91]:
# Or load data for specific year(s)
# df = load_data_by_year(2025)
# df = load_data_by_year([2024, 2025])


In [92]:
# Get summary statistics
get_summary(df)


RESUMEN DEL DATASET
Total de registros: 46,014
Códigos BDNS únicos: 46,014

Rango de fechas: 2023-05-31 00:00:00 a 2024-03-06 00:00:00

Años disponibles: [2023, 2024]

ESTADÍSTICAS GENERALES DE PRESUPUESTO
--------------------------------------------------------------------------------
  Total: €58,495,272,436.34
  Media: €1,271,249.46
  Mediana: €15,400.00
  Mínimo: €0.00
  Máximo: €4,134,000,000.00

ESTADO GENERAL
--------------------------------------------------------------------------------
  Abiertas: 3,120
  Cerradas: 42,894

ANÁLISIS POR AÑO

AÑO 2023
Registros: 35,942
Códigos BDNS únicos: 35,942

Rango de fechas: 2023-05-31 00:00:00 a 2023-12-31 00:00:00

Presupuesto:
  Total: €36,643,469,200.69
  Media: €1,019,516.70
  Mediana: €16,298.00
  Mínimo: €0.00
  Máximo: €4,134,000,000.00

Estado:
  Abiertas: 2,566
  Cerradas: 33,376

Top 5 regiones:
  1. ES511 - Barcelona: 2,553
  2. ES523 - Valencia / València: 1,626
  3. ES521 - Alicante / Alacant: 1,408
  4. ES51 - CATALUÑA: 1,3

### Example 1: Search for specific BDNS number


In [93]:
# Search for BDNS 865179
result = filter_by_bdns(df, 865179)
display_results(result)


No results found.


### Example 2: Search by keyword in descripcion


In [106]:
# Search for "educación" in description
result = search_descripcion(df, "inclusivo")
display_results(result)


Found 25 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
2283,702332,2023-06-12,Subvenciones para la creación espacios más par...,119247.32,PARLA,ES30 - COMUNIDAD DE MADRID,False
3277,703361,2023-06-15,SUBVENCIÓN NOMINATIVA A FAVOR DE LA FUNDACIÓN ...,92137.74,CABILDO INSULAR DE GRAN CANARIA,ES705 - Gran Canaria,False
7313,707609,2023-07-10,SUBVENCIÓN DIRECTA ASOCIACIÓN “A TODA VELA” PA...,8000.00,DIPUTACIÓN PROV. DE ALMERÍA,ES611 - Almería,False
9545,709883,2023-07-25,"FONS VALENCIÀ PER LA SOLIDARITAT PROYECTO ""OTA...",22769.89,QUART DE POBLET,ES523 - Valencia / València,False
15485,716075,2023-09-11,Ellas+ Marruecos 2023. Dinámicas territoriales...,300000.00,ESTADO,XXXX - TODO EL MUNDO,False
15986,716587,2023-09-14,"RES PRESIDENTA IMD NÚM. 426, 31/08/23, DE CONC...",40000.00,"PALMAS DE GRAN CANARIA, LAS",ES705 - Gran Canaria,False
16833,717453,2023-09-20,"Orden de la Consejera de Presidencia, Interior...",12500.00,ARAGÓN,ES24 - ARAGON,False
18770,719442,2023-10-03,CONVENIO DE COLABORACIÓN CON LA ASOCIACIÓN INC...,10000.00,CÓRDOBA,ES613 - Córdoba,False
20490,721216,2023-10-13,AYUDA EXCEPCIONAL AYUNYAMIENTO DE LA GINETA PA...,2000.00,DIPUTACIÓN PROV. DE ALBACETE,ES421 - Albacete,False
20707,721446,2023-10-16,Resolución de Presidencia de 13 de octubre de ...,10412.06,DIPUTACIÓN PROV. DE MÁLAGA,ES617 - Málaga,False



... and 15 more rows


### Example 3: Filter by region


In [95]:
# Filter by Comunidad Valenciana
result = filter_by_region(df, "VALENCIANA")
display_results(result)


Found 984 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
17,700017,2023-05-31,"Resolución de 30 de mayo de 2023, del presiden...",2191000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
87,700088,2023-06-01,"Resolución de la consellera de Agricultura, De...",300000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
130,700132,2023-06-01,Elaboración de instrumentos de planeamiento de...,500000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
169,700171,2023-06-01,"CONVENIO COLABORACION ENTRE LA G., A TRAVES DE...",10000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
177,700179,2023-06-01,CONVENIO CON AYUNTAMIENTO DE AIELO DE MALFERIT...,40000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
180,700182,2023-06-01,Resolución de la Consellería de Economía Soste...,40000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
183,700185,2023-06-01,Resolución de la Consellería de Economía Soste...,30000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
204,700206,2023-06-01,RESOL CONSELLER ECONOMIA SOSTENIBLE A FAVOR DE...,50000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
213,700215,2023-06-01,"CONVENIO DE COLABORACIÓN ENTRE LA GENERALITAT,...",50000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
217,700219,2023-06-01,"CONVENIO CON EL AYUNTAMIENTO DE RELLEU, PARA L...",60000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False



... and 974 more rows


### Example 4: Filter by budget range


In [105]:
# Find convocatorias with budget over €1,000,000
result = filter_by_budget(df, min_budget=1000000000)
display_results(result)


Found 8 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
27057,727967,2023-11-17,Selección de operaciones para su cofinanciació...,1.445207e+09,ESTADO,ES - ESPAÑA,False
34377,735457,2023-12-26,Línea de avales ICO para la cobertura por cuen...,4.134000e+09,ESTADO,ES - ESPAÑA,False
40055,741267,2024-01-26,"Resolución de 25 de enero de 2024,de la D.G.de...",1.534355e+09,ANDALUCÍA,ES61 - ANDALUCIA,False
41202,742465,2024-02-05,REAFIANZAMIENTO PROGRAMA DE AVALES PARA PYMES ...,1.600000e+09,ESTADO,ES - ESPAÑA,False
41208,742471,2024-02-05,REAFIANZAMIENTO DEL PROGRAMA DE AVALES PARA PY...,1.600000e+09,ESTADO,ES - ESPAÑA,False
41213,742476,2024-02-05,REAFIANZAMIENTO PROGRAMA DE AVALES PARA PYMES ...,1.600000e+09,ESTADO,ES - ESPAÑA,False
41232,742495,2024-02-05,MARCO NACIONAL TEMPORAL RELATIVO A LAS MEDIDAS...,1.600000e+09,ESTADO,ES - ESPAÑA,False
41252,742515,2024-02-05,REAFIANZAMIENTO PROGRAMA DE AVALES PARA PYMES ...,1.600000e+09,ESTADO,ES - ESPAÑA,False


### Example 5: Filter by date range


In [97]:
# Find convocatorias received in October 2025
result = filter_by_date_range(df, 
                               date_field='fechaRecepcion',
                               start_date='2025-10-01',
                               end_date='2025-10-31')
display_results(result)


No results found.


### Example 6: Filter by institution


In [98]:
# Find convocatorias from Comunitat Valenciana institutions
result = filter_by_institution(df, nivel1="VALENCIANA")
display_results(result)


Found 651 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
17,700017,2023-05-31,"Resolución de 30 de mayo de 2023, del presiden...",2191000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
87,700088,2023-06-01,"Resolución de la consellera de Agricultura, De...",300000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
130,700132,2023-06-01,Elaboración de instrumentos de planeamiento de...,500000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
169,700171,2023-06-01,"CONVENIO COLABORACION ENTRE LA G., A TRAVES DE...",10000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
177,700179,2023-06-01,CONVENIO CON AYUNTAMIENTO DE AIELO DE MALFERIT...,40000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
180,700182,2023-06-01,Resolución de la Consellería de Economía Soste...,40000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
183,700185,2023-06-01,Resolución de la Consellería de Economía Soste...,30000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
204,700206,2023-06-01,RESOL CONSELLER ECONOMIA SOSTENIBLE A FAVOR DE...,50000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
213,700215,2023-06-01,"CONVENIO DE COLABORACIÓN ENTRE LA GENERALITAT,...",50000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False
217,700219,2023-06-01,"CONVENIO CON EL AYUNTAMIENTO DE RELLEU, PARA L...",60000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,False



... and 641 more rows


### Example 7: Filter by sector


In [99]:
# Find education-related convocatorias
result = filter_by_sector(df, sector_desc="Educación")
display_results(result)


Found 6,058 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
2,700002,2023-05-31,Orde axudas para o fomento da gratuidade da at...,43285939.0,GALICIA,ES11 - GALICIA,False
6,700006,2023-05-31,Decreto Foral 96/2023 del Diputado General que...,100000.0,DIPUTACIÓN FORAL DE BIZKAIA,ES213 - Bizkaia,False
12,700012,2023-05-31,"Resolución de 30 de mayo de 2023, del Rectorad...",2000.0,OTROS,ES - ESPAÑA,False
25,700025,2023-05-31,"Resolución Rectoral 112/2023, de la Universida...",3000.0,OTROS,XXXX - TODO EL MUNDO,False
30,700031,2023-05-31,Subvención nominativa L'Escola La Càpsula,20000.0,SANT BOI DE LLOBREGAT,ES511 - Barcelona,False
50,700051,2023-05-31,"Resolución de 6 de junio de 2023,de la D.G. de...",336535.0,ANDALUCÍA,ES61 - ANDALUCIA,False
68,700069,2023-05-31,Orden de concesión directa de subvención al Ay...,119182.5,CANARIAS,ES70 - CANARIAS,False
72,700073,2023-06-01,PREMIOS EXTRAORDINARIOS DE FORMACIÓN PROFESSIO...,19000.0,ILLES BALEARS,ES53 - ILLES BALEARS,True
93,700094,2023-06-01,CONVENIO DE COLABORACIÓN ENTRE LA DIPUTACIÓN D...,15000.0,DIPUTACIÓN PROV. DE PALENCIA,ES414 - Palencia,True
95,700096,2023-06-01,III Edición de los premios a los mejores traba...,4000.0,COMUNIDAD FORAL DE NAVARRA,ES22 - COMUNIDAD FORAL DE NAVARRA,False



... and 6,048 more rows


### Example 8: Filter by open status


In [100]:
# Find currently open convocatorias
result = filter_by_status(df, abierto=True)
display_results(result)


Found 3,120 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
24,700024,2023-05-31,Convenio de colaboración con la Asociación Cul...,8000.00,"PORRIÑO, O",ES114 - Pontevedra,True
72,700073,2023-06-01,PREMIOS EXTRAORDINARIOS DE FORMACIÓN PROFESSIO...,19000.00,ILLES BALEARS,ES53 - ILLES BALEARS,True
87,700088,2023-06-01,"Resolución de la consellera de Agricultura, De...",300000.00,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
89,700090,2023-06-01,Convenio de colaboración con la Fundación Giaf...,24801.39,MIRANDA DE EBRO,ES412 - Burgos,True
93,700094,2023-06-01,CONVENIO DE COLABORACIÓN ENTRE LA DIPUTACIÓN D...,15000.00,DIPUTACIÓN PROV. DE PALENCIA,ES414 - Palencia,True
102,700104,2023-06-01,Subvención Asociación de Criadores de Raza Bov...,6500.00,DIPUTACIÓN PROV. DE HUESCA,ES241 - Huesca,True
107,700109,2023-06-01,Subvención Ayuntamiento de Castejón de Sos. Me...,26852.50,DIPUTACIÓN PROV. DE HUESCA,ES241 - Huesca,True
113,700115,2023-06-01,Subvención Ayuntamiento de Bailo. Pavimentació...,37836.59,DIPUTACIÓN PROV. DE HUESCA,ES241 - Huesca,True
117,700119,2023-06-01,Subvención Asociación familias y mujeres del m...,6000.00,DIPUTACIÓN PROV. DE HUESCA,ES241 - Huesca,True
121,700123,2023-06-01,Subvención FUNDACIÓN VALENTIA. Adquisición de ...,14151.65,DIPUTACIÓN PROV. DE HUESCA,ES241 - Huesca,True



... and 3,110 more rows


### Example 9: Combine multiple filters


In [101]:
# Find open education convocatorias in Comunidad Valenciana with budget > €100,000
result = df.copy()
result = filter_by_status(result, abierto=True)
result = filter_by_sector(result, sector_desc="Educación")
result = filter_by_region(result, "VALENCIANA")
result = filter_by_budget(result, min_budget=100000)

display_results(result)


Found 4 results



,codigoBDNS,fechaRecepcion,descripcion,presupuestoTotal,organo_nivel1,region_descripcion,abierto
710,700721,2023-06-02,"Res. 2/06/2023 de la Consell. de Educación, po...",1800000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
3750,703845,2023-06-19,"RESOLUCIÓ de XX de juny de 2023, de la Consell...",600000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
5878,706097,2023-06-30,"CONVENI ENTRE LA UNIVERSITAT DE VALÈNCIA, LA C...",745639.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True
8140,708447,2023-07-14,CONVENIO PARA LA CONSTRUCCION DEL CENTRO DE IN...,3456000.0,COMUNITAT VALENCIANA,ES52 - COMUNIDAD VALENCIANA,True


### Example 10: View specific columns


In [102]:
# Display custom columns
custom_columns = [
    'codigoBDNS', 
    'descripcion', 
    'presupuestoTotal', 
    'fechaInicioSolicitud',
    'fechaFinSolicitud',
    'urlBasesReguladoras'
]

result = filter_by_status(df, abierto=True)
display_results(result, max_rows=5, columns=custom_columns)


Found 3,120 results



,codigoBDNS,descripcion,presupuestoTotal,fechaInicioSolicitud,fechaFinSolicitud,urlBasesReguladoras
24,700024,Convenio de colaboración con la Asociación Cul...,8000.00,None,None,https://oporrino.sedelectronica.gal/transparen...
72,700073,PREMIOS EXTRAORDINARIOS DE FORMACIÓN PROFESSIO...,19000.00,None,None,http://boib.caib.es/pdf/2009100/mp41.pdf
87,700088,"Resolución de la consellera de Agricultura, De...",300000.00,None,None,https://dogv.gva.es/datos/2022/12/31/pdf/2022_...
89,700090,Convenio de colaboración con la Fundación Giaf...,24801.39,None,None,https://www.mirandadeebro.es/Miranda/Ayuntamie...
93,700094,CONVENIO DE COLABORACIÓN ENTRE LA DIPUTACIÓN D...,15000.00,None,None,https://www.diputaciondepalencia.es/system/fil...



... and 3,115 more rows


### Example 11: Export filtered results


In [103]:
# Export filtered results to CSV
# result = filter_by_region(df, "VALENCIANA")
# result.to_csv('valenciana_convocatorias.csv', index=False)
# print(f"Exported {len(result)} rows to valenciana_convocatorias.csv")


## Your Custom Queries

Use the cells below for your own custom queries and analysis.


In [104]:
# Your custom query here
